In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import subprocess
from subprocess import check_output
try:
    print(check_output(["ls", "../v2"]).decode("utf8"))
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
# Any results you write to the current directory are saved as output.
import tensorflow as tf

frame
models
oldData
video



/Users/user/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video
video_files = os.listdir("../v2/video/")
frame_files = os.listdir("../v2/frame/")

# Global variables
num_labels = 4716

In [4]:
def createTargetVec(labels):
    out = np.zeros((1, num_labels))
    for label in labels:
        out[0,label] = 1
    return out

In [5]:
def get_dataset(frames, samples=5, k=0):
    rgb_input = np.empty((samples, 100, 1024))
    audio_input = np.empty((samples, 100, 128))
    label_output = np.empty((samples, num_labels))
    sess = tf.InteractiveSession()
    for example in tf.python_io.tf_record_iterator(frames):        
        tf_seq_example = tf.train.SequenceExample.FromString(example)
        labels = tf_seq_example.context.feature['labels'].int64_list.value
        rgb_frame = np.zeros((100, 1024))
        audio_frame = np.zeros((100, 128))
        for i in range(100):
            rgb_frame[i] = tf.cast(tf.decode_raw(
                   tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                          ,tf.float32).eval()
            audio_frame[i] = tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval()
        rgb_input[k] = rgb_frame
        audio_input[k] = audio_frame
        label_output[k] = createTargetVec(labels[:])
        k += 1
        progress = (k / samples) * 100
        if int(progress) % 10 == 0:
            print("Progress", progress, "%")
        if k >= samples:
            break
    sess.close()
    return audio_input, rgb_input, label_output

In [6]:
# video_lvl_record = "../v2/video/train-1.tfrecord"
frame_lvl_record = "../v2/frame/train4f.tfrecord"

audio_input, rgb_input, label_output = get_dataset(frame_lvl_record)

X_train_rgb, X_test_rgb, _, _ = train_test_split(rgb_input, label_output, test_size=0.2, random_state=42)
X_train_audio, X_test_audio, y_train, y_test = train_test_split(audio_input, label_output, test_size=0.2, random_state=42)

Progress 20.0 %
Progress 40.0 %
Progress 60.0 %
Progress 80.0 %
Progress 100.0 %


In [7]:
print(len(X_train_rgb), 'train rgb')
print(len(X_test_rgb), 'test rgb')
print("")
print(len(X_train_audio), 'train sequences')
print(len(X_test_audio), 'test sequences')
print("")
print('X_train_rgb shape:', X_train_rgb.shape)
print('X_test_rgb shape:', X_test_rgb.shape)
print("")
print('X_train_audio shape:', X_train_audio.shape)
print('X_test_audio shape:', X_test_audio.shape)
print("")
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

4 train rgb
1 test rgb

4 train sequences
1 test sequences

X_train_rgb shape: (4, 100, 1024)
X_test_rgb shape: (1, 100, 1024)

X_train_audio shape: (4, 100, 128)
X_test_audio shape: (1, 100, 128)

y_train shape: (4, 4716)
y_test shape: (1, 4716)


## Lets take a look at the rgb data

In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [9]:
rgb_input

array([[[100.,  54.,  84., ...,  46.,  10., 132.],
        [105.,   0.,  91., ..., 153.,  27., 103.],
        [ 93.,  24.,  95., ...,  84., 113., 130.],
        ...,
        [ 90.,  35., 122., ..., 190., 155., 147.],
        [ 86.,  26., 112., ..., 129., 100., 126.],
        [ 81.,  12., 120., ..., 222., 114., 140.]],

       [[ 65., 186., 255., ..., 151.,  58., 141.],
        [ 51., 198., 255., ..., 144.,  63., 167.],
        [ 89., 175., 255., ..., 255.,  80., 119.],
        ...,
        [148., 169., 255., ...,  13., 158., 101.],
        [159., 156., 203., ..., 103., 186.,  56.],
        [118., 176., 207., ..., 133., 156., 106.]],

       [[  0.,  85., 148., ..., 124., 135., 133.],
        [  0.,  62., 104., ..., 116., 111., 128.],
        [  0.,  75., 103., ...,  74.,  26., 176.],
        ...,
        [144., 172.,  86., ..., 244.,  72.,  95.],
        [210., 153., 100., ..., 255., 200., 189.],
        [200., 160.,  99., ..., 194., 220., 195.]],

       [[161.,  57.,  93., ...,  44.,

In [10]:
rgb_input.shape

(5, 100, 1024)

In [11]:
rgb_input[0]

array([[100.,  54.,  84., ...,  46.,  10., 132.],
       [105.,   0.,  91., ..., 153.,  27., 103.],
       [ 93.,  24.,  95., ...,  84., 113., 130.],
       ...,
       [ 90.,  35., 122., ..., 190., 155., 147.],
       [ 86.,  26., 112., ..., 129., 100., 126.],
       [ 81.,  12., 120., ..., 222., 114., 140.]])

In [12]:
rgb_input[0].shape

(100, 1024)

In [19]:
print('The first video has %d frames' %len(rgb_input[0]))

The first video has 100 frames


In [18]:
rgb_input[0][0]

array([100.,  54.,  84., ...,  46.,  10., 132.])

In [27]:
rgb_input[0][0].shape

(1024,)

In [35]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import glob
import cv2

In [38]:
# x_data = np.array( [np.array(cv2.imread(rgb_input[0][0])) for i in range(len(rgb_input[0][0]))] )
# print (x_data.shape)

## Now anylyze the Audio data
** Since we're given sequential data, most likely I'll use an LSTM or GRU**